In [12]:
import os
import random
import pandas as pd
from pathlib import Path
import numpy as np
import torch
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable

from scipy.ndimage import gaussian_filter1d
from scipy.stats import pointbiserialr
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.metrics import (
    accuracy_score, 
    roc_auc_score, 
    roc_curve, 
)

from side_info_decoding.utils import (
    set_seed, 
    load_data_from_pids, 
    sliding_window_over_trials
)
from side_info_decoding.reduced_rank import (
    Reduced_Rank_Model, 
    train_single_task, 
    model_eval,
    Multi_Task_Reduced_Rank_Model,
    train_multi_task,
)

import pymc3 as pm
from hmmlearn import vhmm
from side_info_decoding.bmm_hmm import (
    BetaProcess, Constrained_BMM_HMM, posterior_inference
)
from side_info_decoding.viz import plot_multi_session_hmm_results, plot_bmm_hmm_results

seed = 666
set_seed(seed)

In [9]:
class ReducedRankRegression(torch.nn.Module):    
    def __init__(self, n_c, n_t, n_r):
        super(ReducedRankRegression, self).__init__()
        self.U = torch.nn.Parameter(torch.randn(n_c, n_r))
        self.V = torch.nn.Parameter(torch.randn(n_r, n_t))
        self.b = torch.nn.Parameter(torch.randn(1,))
        self.double()
        
    def forward(self, x):
        self.beta = self.U @ self.V
        out = torch.einsum("ct,kct->kt", self.beta, x)
        out += self.b
        return out
    
def train_rrr(
    X, 
    Y, 
    train,
    test,
    n_r = 10,
    learning_rate=1e-3,
    n_epochs=10000,
    ):
    
    _, n_c, n_t = X.shape
    rrr = ReducedRankRegression(n_c, n_t, n_r)
    optimizer = torch.optim.Adam(rrr.parameters(), lr=learning_rate, weight_decay=1e-3)
    criterion = torch.nn.MSELoss()
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    train_x, test_x = X[train], X[test]
    train_y, test_y = Y[train], Y[test]
    
    losses = []
    for epoch in tqdm(range(n_epochs), desc="Train RRR:"):
        optimizer.zero_grad()
        y_pred = rrr(train_x)
        loss = criterion(y_pred, train_y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return rrr, losses

In [10]:
def time_bin_wise_metrics(y_true, y_pred):
    r2 = r2_score(y_true.flatten(), y_pred.flatten())
    adj_r2 = explained_variance_score(y_true, y_pred)
    return r2, adj_r2

def sliding_window_over_time(data, window_size=10):
    
    if len(data.shape) == 3:
        n_trials, n_units, n_t_bins = data.shape
        data_window = np.zeros((
            n_trials, n_units, n_t_bins - window_size + 1, window_size
        ))
        for t in range(window_size-1, n_t_bins):
            tmp_window = np.zeros((n_trials, n_units, window_size))
            for d in range(window_size):
                  tmp_window[:,:,d] = data[:,:,t - d]
            data_window[:,:,t-window_size] = tmp_window
    else:
        n_trials, n_t_bins = data.shape
        data_window = np.zeros((n_trials, n_t_bins - window_size + 1))
        for t in range(window_size-1, n_t_bins):
            data_window[:,t-window_size] = data[:,t]
    return data_window

def causal_decoder(train_X, train_Y, test_X, test_Y, verbose=True):
    penalty = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
    
    train_K, C, T, L = train_X.shape
    test_K, _, _, _ = test_X.shape
    
    train_X = train_X.transpose(0,1,-1,2).reshape(train_K, C*L, T).transpose(0,-1,1).reshape((-1, C*L))
    test_X = test_X.transpose(0,1,-1,2).reshape(test_K, C*L, T).transpose(0,-1,1).reshape((-1, C*L))
    train_Y = train_Y.reshape((-1))
    test_Y = test_Y.reshape((-1))
    
    decoder = GridSearchCV(Ridge(), {"alpha": penalty})
    decoder.fit(train_X, train_Y)
    pred_Y = decoder.predict(test_X)
    test_Y = test_Y.reshape(test_K, T)
    pred_Y = pred_Y.reshape(test_K, T)
    
    r2, adj_r2 = time_bin_wise_metrics(test_Y, pred_Y)
    if verbose:
        print(f"r2: {r2:.3f} adj-r2: {adj_r2:.3f}")
    
    return pred_Y, [r2, adj_r2]

In [2]:
%matplotlib inline
plt.rc("figure", dpi=200)
SMALL_SIZE = 10
BIGGER_SIZE = 10
plt.rc('font', size=BIGGER_SIZE)
plt.rc('axes', titlesize=BIGGER_SIZE)
plt.rc('axes', labelsize=BIGGER_SIZE)
plt.rc('axes', linewidth=1)
plt.rc('xtick', labelsize=BIGGER_SIZE)
plt.rc('ytick', labelsize=BIGGER_SIZE)
plt.rc('legend', fontsize=SMALL_SIZE)
plt.rc('figure', titlesize=1)
plt.rcParams['xtick.major.size'] = 3
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.major.size'] = 3
plt.rcParams['ytick.minor.size'] = 3

#### RRR vs. multi-session RRR

In [4]:
# pids = [
#     "dab512bd-a02d-4c1f-8dbc-9155a163efc0",
#     "febb430e-2d50-4f83-87a0-b5ffbb9a4943",
#     "6fc4d73c-2071-43ec-a756-c6c6d8322c8b",
#     "523f8301-4f56-4faf-ab33-a9ff11331118",
#     "143dd7cf-6a47-47a1-906d-927ad7fe9117",
#     '1a60a6e1-da99-4d4e-a734-39b1d4544fad',
#     '0b8ea3ec-e75b-41a1-9442-64f5fbc11a5a',
#     '1e176f17-d00f-49bb-87ff-26d237b525f1',
#     '16799c7a-e395-435d-a4c4-a678007e1550',
#      'ad714133-1e03-4d3a-8427-33fc483daf1a',
#      '31f3e083-a324-4b88-b0a4-7788ec37b191',
# ]

pids = [
    "6fc4d73c-2071-43ec-a756-c6c6d8322c8b",
    "1a60a6e1-da99-4d4e-a734-39b1d4544fad",
    "dab512bd-a02d-4c1f-8dbc-9155a163efc0",
    "febb430e-2d50-4f83-87a0-b5ffbb9a4943",
    "84bb830f-b9ff-4e6b-9296-f458fb41d160",
    "523f8301-4f56-4faf-ab33-a9ff11331118",
    "143dd7cf-6a47-47a1-906d-927ad7fe9117",
]

In [5]:
X_dict, Y_dict = load_data_from_pids(
    pids,
    brain_region="all",
    behavior="prior",
    data_type="all_ks",
    n_t_bins=30,
    prior_path = "/mnt/3TB/yizi/Downloads/ONE/openalyx.internationalbrainlab.org/paper_repro_ephys_data/figure9_10/priors/"
)

pulling data from ibl database ..
eid: dac3a4c1-b666-4de0-87e8-8c514483cacf
pid: 6fc4d73c-2071-43ec-a756-c6c6d8322c8b
number of trials found: 428
found 428 trials from 281.82 to 3112.92 sec.


Compute spike count: 100%|███████████████████| 428/428 [00:02<00:00, 197.43it/s]


pulling data from ibl database ..
eid: 51e53aff-1d5d-4182-a684-aba783d50ae5
pid: 1a60a6e1-da99-4d4e-a734-39b1d4544fad
number of trials found: 450
found 450 trials from 20.56 to 2612.19 sec.


Compute spike count: 100%|████████████████████| 450/450 [00:06<00:00, 65.50it/s]


pulling data from ibl database ..
eid: d23a44ef-1402-4ed7-97f5-47e9a7a504d9
pid: dab512bd-a02d-4c1f-8dbc-9155a163efc0
number of trials found: 367
found 367 trials from 17.56 to 2310.24 sec.


...

Compute spike count: 100%|████████████████████| 688/688 [00:31<00:00, 21.53it/s]


pulling data from ibl database ..
eid: 88224abb-5746-431f-9c17-17d7ef806e6a
pid: 523f8301-4f56-4faf-ab33-a9ff11331118
number of trials found: 397
found 397 trials from 37.78 to 2183.45 sec.


Compute spike count: 100%|████████████████████| 397/397 [00:05<00:00, 78.54it/s]


pulling data from ibl database ..
eid: 3638d102-e8b6-4230-8742-e548cd87a949
pid: 143dd7cf-6a47-47a1-906d-927ad7fe9117
number of trials found: 605
found 605 trials from 32.91 to 4786.28 sec.


Compute spike count: 100%|███████████████████| 605/605 [00:03<00:00, 155.62it/s]


In [6]:
rank = [2, 5, 10, 15]

In [ ]:
d = 3 # half window size
n_epochs = 700

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
skf = KFold(n_splits=5, shuffle=True, random_state=seed)

n_units = []
train_X_dict, test_X_dict, train_Y_dict, test_Y_dict = {}, {}, {}, {}
for pid in pids:
    X, Y = X_dict[pid], Y_dict[pid]
    K, C, T = X.shape
    n_units.append(C)
    X = sliding_window_over_trials(X, half_window_size=d)
    Y = sliding_window_over_trials(Y, half_window_size=d)
    X, Y = torch.tensor(X), torch.tensor(Y)
    train_X_dict.update({pid: [X[train] for train, _ in skf.split(X, Y)]})
    test_X_dict.update({pid: [X[test] for _, test in skf.split(X, Y)]})
    train_Y_dict.update({pid: [Y[train] for train, _ in skf.split(X, Y)]})
    test_Y_dict.update({pid: [Y[test] for _, test in skf.split(X, Y)]})

for R in [10, 15]:
    for fold_idx in range(5):

        print(f"start training fold {fold_idx+1} on {len(pids)} sessions ..")
        train_X_lst = [train_X_dict[pid][fold_idx] for pid in pids]
        test_X_lst = [test_X_dict[pid][fold_idx] for pid in pids]
        train_Y_lst = [train_Y_dict[pid][fold_idx] for pid in pids]
        test_Y_lst = [test_Y_dict[pid][fold_idx] for pid in pids]

        multi_task_rrm = Multi_Task_Reduced_Rank_Model(
            n_tasks=len(pids),
            n_units=n_units, 
            n_t_bins=T, 
            rank=R, 
            half_window_size=d
        )

        # training
        multi_task_rrm, train_losses = train_multi_task(
            model=multi_task_rrm,
            train_dataset=(train_X_lst, train_Y_lst),
            test_dataset=(test_X_lst, test_Y_lst),
            # loss_function=torch.nn.BCELoss(),
            loss_function=torch.nn.MSELoss(),
            learning_rate=1e-2,
            weight_decay=1e-1,
            n_epochs=n_epochs,
        )

        # eval
        test_U, test_V, test_metrics, _ = model_eval(
            multi_task_rrm, 
            train_dataset=(train_X_lst, train_Y_lst),
            test_dataset=(test_X_lst, test_Y_lst),
            behavior="prior"
        )

        save_path = Path("./sensitivity_rank") / "prior" / f"rank_{R}"
        os.makedirs(save_path, exist_ok=True) 
        np.save(save_path/f"fold_{fold_idx}.npy", test_metrics)

start training fold 1 on 7 sessions ..


In [3]:
rank = [2, 5, 10, 15]

In [14]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, explained_variance_score
from scipy.stats import pearsonr
from scipy.ndimage import gaussian_filter1d
from sklearn.model_selection import KFold, GridSearchCV, train_test_split

In [ ]:
R = 2
behavior_type = "wheel_speed"

pids = [
    # "dab512bd-a02d-4c1f-8dbc-9155a163efc0",
    "febb430e-2d50-4f83-87a0-b5ffbb9a4943",
    "523f8301-4f56-4faf-ab33-a9ff11331118",
    "84bb830f-b9ff-4e6b-9296-f458fb41d160",
    '0b8ea3ec-e75b-41a1-9442-64f5fbc11a5a',
]

for pid in pids:
    
    try:
        X_dict, Y_dict = load_data_from_pids(
            [pid],
            brain_region="all",
            behavior=behavior_type,
            data_type="all_ks",
            n_t_bins = 60,
            # align_time_type="feedback_times",
            align_time_type="firstMovement_times",
            t_before=0.2,
            t_after=1.0,
            normalize_input=False
        )
    except:
        print(f"PID {pid} data missing.")
    
    X, Y = X_dict[pid], Y_dict[pid]
    offset = Y.max()
    Y /= offset
    K, C, T = X.shape
    print(f"PID: {pid} # trials: {K} # neurons {C} # time bins {T}.")
    
    baseline_metrics = []
    kf = KFold(n_splits=5, shuffle=True, random_state=666)
    for fold_idx, (train, test) in enumerate(kf.split(X, Y)):
        pass
        
    # baseline
    window_X = sliding_window_over_time(X, window_size=10)
    window_Y = sliding_window_over_time(Y, window_size=10)
    train_X, test_X = window_X[train], window_X[test]
    train_Y, test_Y = window_Y[train], window_Y[test]

    rrr, losses = train_rrr(
        X = X,
        Y = Y, 
        train = train,
        test = test,
        n_r = R,
        learning_rate = 1e-2,
        n_epochs = 1000
    )

    supervised_subspace = rrr.U.detach().numpy()
    subspace_weights = rrr.V.detach().numpy()

    all_proj = X.transpose(0,-1,1) @ supervised_subspace
    weighted_proj = all_proj * subspace_weights.T
    weighted_proj = weighted_proj.transpose(0, -1, 1)

    window_X = sliding_window_over_time(weighted_proj, window_size=10)
    window_Y = sliding_window_over_time(Y, window_size=10)
    train_X, test_X = window_X[train], window_X[test]
    train_Y, test_Y = window_Y[train], window_Y[test]
    rrr_pred, rrr_metric = causal_decoder(train_X, train_Y, test_X, test_Y)

    save_path = Path("./sensitivity_rank") / "wheel_speed" / f"rank_{R}"
    os.makedirs(save_path, exist_ok=True) 
    np.save(save_path/f"{pid}.npy", rrr_metric)

pulling data from ibl database ..
eid: db4df448-e449-4a6f-a0e7-288711e7a75a
pid: febb430e-2d50-4f83-87a0-b5ffbb9a4943
number of trials found: 350
found 350 trials from 34.74 to 2414.77 sec.


Compute spike count: 100%|███████████████████| 350/350 [00:01<00:00, 197.91it/s]


PID: febb430e-2d50-4f83-87a0-b5ffbb9a4943 # trials: 350 # neurons 465 # time bins 60.


Train RRR:: 100%|███████████████████████████| 1000/1000 [05:48<00:00,  2.87it/s]


r2: -0.061 adj-r2: -2.745
pulling data from ibl database ..
eid: 88224abb-5746-431f-9c17-17d7ef806e6a
pid: 523f8301-4f56-4faf-ab33-a9ff11331118
number of trials found: 397
found 397 trials from 38.00 to 2184.27 sec.


Compute spike count: 100%|███████████████████| 397/397 [00:03<00:00, 117.80it/s]


PID: 523f8301-4f56-4faf-ab33-a9ff11331118 # trials: 397 # neurons 505 # time bins 60.


Train RRR:: 100%|███████████████████████████| 1000/1000 [06:10<00:00,  2.70it/s]


r2: -0.009 adj-r2: -0.095
